Imports

In [4]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
torch.cuda.is_available()

True

Download datasets

In [9]:
# add download of ModelNet40 in data/ModelNet40
print('Downloading ModelNet40...')
!wget modelnet.cs.princeton.edu/ModelNet40.zip -P data
print('Extracting ModelNet40...')
!unzip -q data/ModelNet40.zip -d data
!rm data/ModelNet40.zip
print('ModelNet40 done')

--2021-12-12 22:01:08--  http://modelnet.cs.princeton.edu/ModelNet40.zip
Resolving modelnet.cs.princeton.edu (modelnet.cs.princeton.edu)... 128.112.136.61
Connecting to modelnet.cs.princeton.edu (modelnet.cs.princeton.edu)|128.112.136.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2039180837 (1.9G) [application/zip]
Saving to: ‘data/ModelNet40.zip’

ModelNet40.zip      100%[===================>]   1.90G  3.24MB/s    in 45m 0s  

2021-12-12 22:46:08 (738 KB/s) - ‘data/ModelNet40.zip’ saved [2039180837/2039180837]

Extracting ModelNet40...
ModelNet40 done
--2021-12-12 22:47:04--  http://download.microsoft.com/download/2/8/5/28564B23-0828-408F-8631-23B1EFF1DAC8/tsdf.zip
Resolving download.microsoft.com (download.microsoft.com)... 2a02:26f0:3500:595::e59, 2a02:26f0:3500:583::e59, 23.213.164.137
Connecting to download.microsoft.com (download.microsoft.com)|2a02:26f0:3500:595::e59|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Loc

Download 7Scenes from https://drive.google.com/file/d/1XdQ3muo5anFA28ZFch06z_iTEjXxLEQi/view and extract into data/7Scene

In [1]:
from data.SevenScenes import SevenScenes

train_dataset = SevenScenes('train')
test_dataset = SevenScenes('test')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 296
# Get length, which is a call to __len__ function
print(f'Length of val set: {len(test_dataset)}')  # expected output: 57

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Length of train set: 296
Length of val set: 57


In [2]:
train_sample = train_dataset[3]
print(f'Pointcloud A: {train_sample["pointcloud_a"].shape}')
print(f'Pointcloud B: {train_sample["pointcloud_b"].shape}')
print(f'Transformation Matrix: {train_sample["transformation_matrix"]}')

Pointcloud A: torch.Size([2048, 3])
Pointcloud B: torch.Size([2048, 3])
Transformation Matrix: tensor([[ 0.9995, -0.0056,  0.0300,  0.0450],
        [ 0.0051,  0.9998,  0.0172,  0.0559],
        [-0.0301, -0.0170,  0.9994,  0.0333],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])


Run simple configuration

In [1]:
# run simple configuration
import pytorch_lightning as pl

config = {
    'lr_D': 0.01,
    'lr_G': 0.01}

from training.trainer import GenReg_Trainer
from data.ModelNet40 import ModelNet40DataModule
trainer = pl.Trainer(max_epochs=1, gpus=1, overfit_batches=0.1)
modelnet = ModelNet40DataModule(batch_size=16)
modelnet.setup()

trainer.fit(model=GenReg_Trainer(params=config), train_dataloaders=modelnet.train_dataloader(), val_dataloaders=modelnet.val_dataloader())

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | Params
-----------------------------------------
0 | G_model | DataParallel | 5.9 M 
1 | D_model | DataParallel | 1.7 M 
-----------------------------------------
7.6 M     Trainable params
0         Non-trainable params
7.6 M     Total params
30.254    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/lucia/anaconda3/envs/GenReg2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (32) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 160.00 MiB (GPU 0; 5.80 GiB total capacity; 4.20 GiB already allocated; 129.81 MiB free; 4.40 GiB reserved in total by PyTorch)

Visualize results

In [8]:
# add visualization